# Coursera Capston - Applied Data Science

## Gary S. Netherton - June 30, 2020

### This notebook will be used for the capstone project for the Applied Data Science coursework.

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Purpose: 
> I have the opportunity to relocate.  My choice, however, will revolve around the selection of restaurants and medical facilities.  I have lived in
> places such as Chicago that have a plethora of food options.  I have also lived places where my options were limited to Subway, McDonald's, and a couple of 
> teriyaki joints.  As I want this to be my last move - I am nearing retirement - I want to find a comfortable, compatible city to match my wants and needs.

> Additionally, I have medical needs as I have had heart issues in the past, so I need to be near cardiologists and established medical facilities.
> I am currently reviewing job offers from two firms - one is located in Temecula, CA and one is located in Portland, OR.

###          Target Audience(s):
> 1. Me
> 2. Families with children
> 3. Families with family members with health concerns

> *The Foursquare API will enable me to research both dining establishments as well as medical facilities.  I will review both the number of both as well as the*
> *ratings provided by users.  Due to recent changes in the Foursquare API for security purposes, I will not dig into specific users and their comments.*

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# Let me know when the libraries are loaded.

print('Libraries imported.')


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         861 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge

### Data Collection

In [4]:
# Use pandas to import data from both California and Oregon
df_All = pd.read_csv('zip_code_database_CA.csv')

df_All['latitude']=pd.to_numeric(df_All['latitude'])
df_All['longitude']=pd.to_numeric(df_All['longitude'])
df_All['irs_estimated_population_2015']=pd.to_numeric(df_All['irs_estimated_population_2015'])

#df_All.head()
df_CA = df_All.loc[df_All['area_codes']=='951',:]
print(df_CA.shape)
#df_OR = df_All.loc[(df_All['area_codes']=='503' & df_All['area_codes']=='971'),:]
df_OR = df_All.loc[(df_All['area_codes']=='503') | (df_All['area_codes']=='971'), :]
print(df_OR.shape)


(48, 15)
(118, 15)


In [5]:
df_CA.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
38625,91752,STANDARD,0,Mira Loma,Eastvale,NaN,CA,Riverside County,America/Los_Angeles,951,NaN,US,33.99,-117.53,30490
38863,92220,STANDARD,0,Banning,NaN,NaN,CA,Riverside County,America/Los_Angeles,951,NaN,US,33.95,-116.83,26920
39012,92502,PO BOX,0,Riverside,NaN,NaN,CA,Riverside County,America/Los_Angeles,951,NaN,US,33.98,-117.37,979
39019,92509,STANDARD,0,Riverside,Jurupa Valley,Rubidoux,CA,Riverside County,America/Los_Angeles,951,NaN,US,33.99,-117.44,72670
39020,92513,PO BOX,0,Riverside,NaN,NaN,CA,Riverside County,America/Los_Angeles,951,NaN,US,33.91,-117.46,1241


In [6]:
df_OR.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
41134,97002,STANDARD,0,Aurora,NaN,Butteville,OR,Marion County,America/Los_Angeles,503,NaN,US,45.23,-122.80,5420
41136,97004,STANDARD,0,Beavercreek,NaN,NaN,OR,Clackamas County,America/Los_Angeles,503,NaN,US,45.25,-122.46,4270
41141,97009,STANDARD,0,Boring,Damascus,NaN,OR,Clackamas County,America/Los_Angeles,971,NaN,US,45.42,-122.37,8140
41142,97010,PO BOX,0,Bridal Veil,NaN,NaN,OR,Multnomah County,America/Los_Angeles,971,NaN,US,45.58,-122.11,49
41144,97013,STANDARD,0,Canby,NaN,Barlow,OR,Clackamas County,America/Los_Angeles,503,NaN,US,45.21,-122.66,22350


In [7]:
# Oregon Latitude 44.1156, Longitude -120.5148
# California Latitude 37.2461, Longitude -119.6104

In [8]:
# Setup the Foursquare API information

CLIENT_ID = 'BRBDGCKR00L1JCYN5RD3ZDJ2BNOJ5YOVAN3F2KZNVVUVB3VW' # your Foursquare ID
CLIENT_SECRET = '5N1FQNDEEVF0LSK4A1SFM41P4KFSGECE1T23QNK20K1SD4PR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: BRBDGCKR00L1JCYN5RD3ZDJ2BNOJ5YOVAN3F2KZNVVUVB3VW
CLIENT_SECRET:5N1FQNDEEVF0LSK4A1SFM41P4KFSGECE1T23QNK20K1SD4PR


In [9]:
# create map of Temecula using latitude and longitude values

map_temecula = folium.Map(location=[33.4934, -117.14879], zoom_start=10)

# add markers to map
for lat, lng, primCity, popul in zip(df_CA['latitude'], df_CA['longitude'], df_CA['primary_city'], df_CA['irs_estimated_population_2015']):
    label = '{}, {}'.format(primCity, popul)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_temecula)  
    
map_temecula

In [10]:
# create map of Portland using latitude and longitude values

map_portland = folium.Map(location=[45.51179, -122.67563], zoom_start=10)

# add markers to map
for lat, lng, primCity, popul in zip(df_OR['latitude'], df_OR['longitude'], df_OR['primary_city'], df_OR['irs_estimated_population_2015']):
    label = '{}, {}'.format(primCity, popul)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_portland)  
    
map_portland

In [11]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [12]:
# Create function to collect venue information from all neighborhoods in Temecula & Portland
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
# Collect the information from Foursquare for Temecula neighborhoods
temecula_venues = getNearbyVenues(df_CA['primary_city'],
                                   df_CA['latitude'],
                                   df_CA['longitude']
                                  )

Mira Loma
Banning
Riverside
Riverside
Riverside
Riverside
Riverside
Riverside
Riverside
March Air Reserve Base
Riverside
Riverside
Riverside
Lake Elsinore
Lake Elsinore
Lake Elsinore
Aguanga
Anza
Hemet
Homeland
Idyllwild
Moreno Valley
Moreno Valley
Moreno Valley
Moreno Valley
Moreno Valley
Moreno Valley
Moreno Valley
Murrieta
Murrieta
Murrieta
Nuevo
Perris
Perris
Perris
San Jacinto
San Jacinto
Menifee
Sun City
Sun City
Sun City
Temecula
Temecula
Wildomar
Perris
Corona
Corona
Corona


In [14]:
# Collect the information from Foursquare for Portland neighborhoods
portland_venues = getNearbyVenues(df_OR['primary_city'],
                                   df_OR['latitude'],
                                   df_OR['longitude']
                                  )

Aurora
Beavercreek
Boring
Bridal Veil
Canby
Clackamas
Colton
Columbia City
Corbett
Donald
Eagle Creek
Estacada
Fairview
Gervais
Government Camp
Lake Oswego
Lake Oswego
Marylhurst
Molalla
Mulino
Warren
Deer Island
Sandy
Scappoose
Troutdale
Beaverton
Beaverton
Beaverton
Beaverton
Happy Valley
Damascus
Arch Cape
Banks
Beaver
Buxton
Cannon Beach
Carlton
Dundee
Forest Grove
Gales Creek
Garibaldi
Hammond
Hebo
Hillsboro
Manning
Lafayette
Manzanita
Oceanside
Pacific City
Sherwood
Netarts
Timber
Tolovana Park
Warrenton
Wheeler
Neskowin
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Salem
Salem
Keizer
Salem
Salem
Salem
Salem
Salem
Salem
Salem
Salem
Dallas
Detroit
Grand Ronde
Inde

In [15]:
portland_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aurora,45.23,-122.80,Rome Transportation LLC,45.231671,-122.801974,Building
1,Aurora,45.23,-122.80,XFINITY Store by Comcast,45.228450,-122.803878,Business Service
2,Aurora,45.23,-122.80,Sunrise Custom Painting,45.225655,-122.800477,Paintball Field
3,Bridal Veil,45.58,-122.11,Cousin's Restaurant & Saloon,45.579640,-122.111006,American Restaurant
4,Canby,45.21,-122.66,Tmk Creamery,45.212623,-122.661790,Cheese Shop


In [16]:
temecula_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mira Loma,33.99,-117.53,Sameday Gate Repair Eastvale,33.993224,-117.528763,Home Service
1,Banning,33.95,-116.83,Malki Museum,33.949280,-116.825279,Museum
2,Riverside,33.98,-117.37,The Old Spaghetti Factory,33.980494,-117.367675,Italian Restaurant
3,Riverside,33.98,-117.37,Riverside Municipal Auditorium,33.982062,-117.370789,Performing Arts Venue
4,Riverside,33.98,-117.37,Romano's Riverside,33.981286,-117.372714,Italian Restaurant


In [17]:
# Number of venues returned for each neighborhood
temecula_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Banning,1,1,1,1,1,1
Corona,4,4,4,4,4,4
Hemet,2,2,2,2,2,2
Homeland,1,1,1,1,1,1
Lake Elsinore,16,16,16,16,16,16
March Air Reserve Base,2,2,2,2,2,2
Menifee,2,2,2,2,2,2
Mira Loma,1,1,1,1,1,1
Moreno Valley,33,33,33,33,33,33


In [18]:
# Number of venues returned for each neighborhood
portland_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arch Cape,1,1,1,1,1,1
Aurora,3,3,3,3,3,3
Beaverton,208,208,208,208,208,208
Bridal Veil,1,1,1,1,1,1
Canby,1,1,1,1,1,1
Cannon Beach,49,49,49,49,49,49
Clackamas,4,4,4,4,4,4
Columbia City,4,4,4,4,4,4
Damascus,8,8,8,8,8,8


In [19]:
print('There are {} uniques categories for Temecula.'.format(len(temecula_venues['Venue Category'].unique())))
print('There are {} uniques categories for Portland.'.format(len(portland_venues['Venue Category'].unique())))


There are 118 uniques categories for Temecula.
There are 232 uniques categories for Portland.


In [20]:
# one hot encoding - Temecula
temecula_onehot = pd.get_dummies(temecula_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
temecula_onehot['Neighborhood'] = temecula_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [temecula_onehot.columns[-1]] + list(temecula_onehot.columns[:-1])
temecula_onehot = temecula_onehot[fixed_columns]
print(temecula_onehot.shape)
temecula_onehot.head()

(330, 119)


,Neighborhood,ATM,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,Bakery,Bank,Bar,Baseball Field,Beer Garden,Big Box Store,Bike Trail,Breakfast Spot,Bubble Tea Shop,Burger Joint,Café,Cajun / Creole Restaurant,Campground,Candy Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Food,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Home Service,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Intersection,Italian Restaurant,Japanese Restaurant,Kids Store,Lake,Latin American Restaurant,Laundry Service,Lawyer,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Men's Store,Mexican Restaurant,Mobile Phone Shop,Mountain,Movie Theater,Museum,Music Venue,Office,Park,Performing Arts Venue,Pizza Place,Pub,Rental Car Location,Rental Service,Resort,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Spa,Sporting Goods Shop,Steakhouse,Supplement Shop,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Toy / Game Store,Train Station,Tree,Vape Store,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Mira Loma,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Banning,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Riverside,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Riverside,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Riverside,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
# one hot encoding - Portland
portland_onehot = pd.get_dummies(portland_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
portland_onehot['Neighborhood'] = portland_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [portland_onehot.columns[-1]] + list(portland_onehot.columns[:-1])
portland_onehot = portland_onehot[fixed_columns]
print(portland_onehot.shape)
portland_onehot.head()

(1391, 233)


,Neighborhood,ATM,Adult Boutique,Airport,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Eye Doctor,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Fondue Restaurant,Food,Food Court,Food Truck,Forest,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Moroccan Restaurant,Motel,Mountain,Movie Theater,Moving Target,Music Store,Nail Salon,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Post Office,Pub,RV Park,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,River,Rock Club,Roof Deck,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Street Food Gathering,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Vineyard,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Aurora,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Aurora,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [22]:
temecula_grouped = temecula_onehot.groupby('Neighborhood').mean().reset_index()
print('Temecula:')
print(temecula_grouped.shape)
portland_grouped = portland_onehot.groupby('Neighborhood').mean().reset_index()
print("Portland:")
print(portland_grouped.shape)

Temecula:
(15, 119)
Portland:
(30, 233)


In [23]:
num_top_venues = 5

for hood in temecula_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = temecula_grouped[temecula_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Banning----
                   venue  freq
0                 Museum   1.0
1                    ATM   0.0
2                 Market   0.0
3            Pizza Place   0.0
4  Performing Arts Venue   0.0


----Corona----
               venue  freq
0  Convenience Store   1.0
1                ATM   0.0
2     Massage Studio   0.0
3                Pub   0.0
4        Pizza Place   0.0


----Hemet----
            venue  freq
0  Sandwich Place   0.5
1    Liquor Store   0.5
2             ATM   0.0
3  Massage Studio   0.0
4     Pizza Place   0.0


----Homeland----
                   venue  freq
0           Home Service   1.0
1                    ATM   0.0
2                 Market   0.0
3            Pizza Place   0.0
4  Performing Arts Venue   0.0


----Lake Elsinore----
                  venue  freq
0        Sandwich Place  0.12
1  Fast Food Restaurant  0.12
2                   Bar  0.06
3           Pizza Place  0.06
4     Convenience Store  0.06


----March Air Reserve Base----
                 

In [24]:

for hood in portland_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp1 = portland_grouped[portland_grouped['Neighborhood'] == hood].T.reset_index()
    temp1.columns = ['venue','freq']
    temp1 = temp1.iloc[1:]
    temp1['freq'] = temp1['freq'].astype(float)
    temp1 = temp1.round({'freq': 2})
    print(temp1.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arch Cape----
           venue  freq
0         Tunnel   1.0
1            ATM   0.0
2           Park   0.0
3  Movie Theater   0.0
4  Moving Target   0.0


----Aurora----
              venue  freq
0          Building  0.33
1  Business Service  0.33
2   Paintball Field  0.33
3               ATM  0.00
4              Park  0.00


----Beaverton----
                venue  freq
0   Korean Restaurant  0.06
1         Coffee Shop  0.05
2      Sandwich Place  0.04
3       Grocery Store  0.04
4  Mexican Restaurant  0.04


----Bridal Veil----
                   venue  freq
0    American Restaurant   1.0
1                    ATM   0.0
2  Performing Arts Venue   0.0
3          Moving Target   0.0
4            Music Store   0.0


----Canby----
                   venue  freq
0            Cheese Shop   1.0
1                    ATM   0.0
2  Performing Arts Venue   0.0
3          Moving Target   0.0
4            Music Store   0.0


----Cannon Beach----
                 venue  freq
0  American Restauran

In [25]:

# Insert the most common venues data into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:

# The following code creates a new dataframe that will display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = temecula_grouped['Neighborhood']

for ind in np.arange(temecula_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(temecula_grouped.iloc[ind, :], num_top_venues)

print('For Temecula: ')
neighborhoods_venues_sorted.head(10)

For Temecula: 


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Banning,Museum,Women's Store,Fried Chicken Joint,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market
1,Corona,Convenience Store,Frozen Yogurt Shop,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant
2,Hemet,Liquor Store,Sandwich Place,Construction & Landscaping,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market
3,Homeland,Home Service,Frozen Yogurt Shop,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant
4,Lake Elsinore,Fast Food Restaurant,Sandwich Place,Coffee Shop,Bar,Pizza Place,Mountain,Discount Store,Campground,Cajun / Creole Restaurant,Fried Chicken Joint
5,March Air Reserve Base,Coffee Shop,Fast Food Restaurant,Women's Store,Construction & Landscaping,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop
6,Menifee,Mobile Phone Shop,Garden Center,Women's Store,Fried Chicken Joint,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop
7,Mira Loma,Home Service,Frozen Yogurt Shop,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant
8,Moreno Valley,Mexican Restaurant,Pizza Place,Grocery Store,Ice Cream Shop,Fried Chicken Joint,Filipino Restaurant,Asian Restaurant,Automotive Shop,Bakery,Fast Food Restaurant
9,Murrieta,Mexican Restaurant,Pizza Place,Italian Restaurant,Music Venue,Coffee Shop,Dessert Shop,Sandwich Place,Office,Frozen Yogurt Shop,Baseball Field


In [27]:
# The following code creates a new dataframe that will display the top 10 venues for each neighborhood
#num_top_venues = 10

indicators1 = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns1 = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns1.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns1.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted1 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted1['Neighborhood'] = portland_grouped['Neighborhood']

for ind in np.arange(portland_grouped.shape[0]):
    neighborhoods_venues_sorted1.iloc[ind, 1:] = return_most_common_venues(portland_grouped.iloc[ind, :], num_top_venues)

print('For Portland: ')
neighborhoods_venues_sorted1.head(10)

For Portland: 


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arch Cape,Tunnel,Yoga Studio,Doctor's Office,Food,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1,Aurora,Paintball Field,Building,Business Service,Yoga Studio,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
2,Beaverton,Korean Restaurant,Coffee Shop,Sandwich Place,Grocery Store,Mexican Restaurant,Bakery,Furniture / Home Store,Wings Joint,Sushi Restaurant,Department Store
3,Bridal Veil,American Restaurant,Yoga Studio,Dog Run,Food Court,Food,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
4,Canby,Cheese Shop,Yoga Studio,Dog Run,Food Court,Food,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
5,Cannon Beach,American Restaurant,Gift Shop,Restaurant,Art Gallery,Toy / Game Store,Candy Store,Park,Pizza Place,Shopping Mall,Fish & Chips Shop
6,Clackamas,Bar,Brewery,Clothing Store,Basketball Court,Donut Shop,Food Court,Food,Fondue Restaurant,Flower Shop,Fish & Chips Shop
7,Columbia City,Post Office,History Museum,Beach,Park,Yoga Studio,Dog Run,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant
8,Damascus,Construction & Landscaping,Coffee Shop,Pub,Business Service,Grocery Store,Sandwich Place,Home Service,Ice Cream Shop,Creperie,Dog Run
9,Fairview,Recreation Center,Business Service,Park,Yoga Studio,Dog Run,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [28]:

# set number of clusters
kclusters = 5

temecula_grouped_clustering = temecula_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(temecula_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 1, 3, 1, 1, 4, 3, 1, 1])

In [43]:
# add clustering labels for new dataframe to include the cluster and its top 10 venues for Temecula
# currently commented out since I have run this cell so many times
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
# neighborhoods_venues_sorted.head()
temecula_merged = df_CA[['primary_city', 'latitude', 'longitude']]
temecula_merged.rename(columns = {'primary_city':'Neighborhood'}, inplace = True)
temecula_merged.head()


# merge temecula_grouped with df_CA to add latitude/longitude for each neighborhood
neighborhoods_venues_sorted = neighborhoods_venues_sorted.join(temecula_merged.set_index('Neighborhood'), on='Neighborhood')
neighborhoods_venues_sorted.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,latitude,longitude
0,2,Banning,Museum,Women's Store,Fried Chicken Joint,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market,33.95,-116.83
1,0,Corona,Convenience Store,Frozen Yogurt Shop,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant,33.87,-117.56
1,0,Corona,Convenience Store,Frozen Yogurt Shop,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant,33.87,-117.56
1,0,Corona,Convenience Store,Frozen Yogurt Shop,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant,33.75,-117.48
2,1,Hemet,Liquor Store,Sandwich Place,Construction & Landscaping,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market,33.74,-116.97


In [44]:


portland_grouped_clustering = portland_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans2 = KMeans(n_clusters=kclusters, random_state=0).fit(portland_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans2.labels_[0:10]

array([1, 1, 1, 3, 1, 1, 1, 1, 1, 1])

In [63]:
# add clustering labels for new dataframe to include the cluster and its top 10 venues for Portland
# currently commented out since I have run this cell so many times
#neighborhoods_venues_sorted1.insert(0, 'Cluster Labels', kmeans2.labels_)
# neighborhoods_venues_sorted1.head()
portland_merged = df_OR[['primary_city', 'latitude', 'longitude']]
portland_merged.rename(columns = {'primary_city':'Neighborhood'}, inplace = True)
#portland_merged.head()


# merge portland_merged with df_CA to add latitude/longitude for each neighborhood
neighborhoods_venues_sorted1 = neighborhoods_venues_sorted1.join(portland_merged.set_index('Neighborhood'), on='Neighborhood')
neighborhoods_venues_sorted1.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,latitude,longitude
0,1,Arch Cape,Tunnel,Yoga Studio,Doctor's Office,Food,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,NaN,NaN
1,1,Aurora,Paintball Field,Building,Business Service,Yoga Studio,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,NaN,NaN
2,1,Beaverton,Korean Restaurant,Coffee Shop,Sandwich Place,Grocery Store,Mexican Restaurant,Bakery,Furniture / Home Store,Wings Joint,Sushi Restaurant,Department Store,NaN,NaN
3,3,Bridal Veil,American Restaurant,Yoga Studio,Dog Run,Food Court,Food,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,NaN,NaN
4,1,Canby,Cheese Shop,Yoga Studio,Dog Run,Food Court,Food,Fondue Restaurant,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,NaN,NaN


In [54]:
map_clustersCA = folium.Map(location=[33.4934, -117.14879], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]



# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_venues_sorted['latitude'], neighborhoods_venues_sorted['longitude'], \
                                  neighborhoods_venues_sorted['Neighborhood'], neighborhoods_venues_sorted['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clustersCA)
       
map_clustersCA

In [55]:
map_clustersOR = folium.Map(location=[45.51179, -122.67563], zoom_start=11)

# set color scheme for the clusters
# x = np.arange(kclusters)
# ys = [i + x + (i*x)**2 for i in range(kclusters)]
# colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
# rainbow = [colors.rgb2hex(i) for i in colors_array]

# temecula_merged.rename(columns = {'primary_city':'Neighborhood'}, inplace = True)

# add markers to the map
markers_colors2 = []
for lat, lon, poi, cluster in zip(neighborhoods_venues_sorted1['latitude'], neighborhoods_venues_sorted1['longitude'], \
                                  neighborhoods_venues_sorted1['Neighborhood'], neighborhoods_venues_sorted1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clustersOR)
       
map_clustersOR

ValueError: Location values cannot contain NaNs, got:
[nan, nan]